In [1]:
import sys
sys.path.append("D:\\WETrak")

import os
import os.path as path
import numpy as np
import pandas as pd
import json
import torch
from glob import glob

import import_ipynb
import branch_adapter.classification.classification_data_utils as data_utils
os.environ['KMP_DUPLICATE_LIB_OK']='True'

importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_data_utils.ipynb


In [2]:
# [0,default]按照classify_num来划分间隔 （每个角度间隔可能不一样）
class classification_data(torch.utils.data.Dataset):
    # output: 
    # emg: [batch,1000,2] min,max: [batch, 8*3]
    def __init__(self, root_path, user_name, split, finger_type_list,classify_num):
        
        self.user_name = user_name
        self.data_path = path.join(root_path,user_name,split)
#         print("datapath:",path.abspath(self.data_path))
        
        self.finger_type_list = finger_type_list
        self.classify_num = classify_num
        
        self.files = glob(path.join(self.data_path,'*'))
        
        self.emg_data_list = []
        self.leap_data_list = []
        
        for file_path in self.files:
            emg_data = pd.read_csv(path.join(file_path,"emg_filtered.csv"),header=None).to_numpy()
            
            leap_data = json.load(open(path.join(file_path,"leap_data_processed.json"),mode='r', encoding='utf-8'))
            leap_data_temp = data_utils.parse_leap_all_finger(leap_data,self.finger_type_list) # [1000,N,4]
            leap_data = leap_data_temp.reshape(len(leap_data_temp),-1) # [1000,N*4]
            
            self.emg_data_list.append(emg_data)
            self.leap_data_list.append(leap_data)
        
        # emg normalize z-score 
        self.emg_data_list = data_utils.emg_z_norm(self.emg_data_list)
        # smooth gt
        self.leap_data_list = data_utils.smooth_gt(self.leap_data_list,self.finger_type_list)
        # 记录每2s窗口的gt的最大值、最小值
        min_vals_list,max_vals_list,_ = data_utils.gt_normalize(self.leap_data_list,finger_type_list)
        self.min_max_vals = np.concatenate((min_vals_list, max_vals_list), axis=1)

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        # emg_data[idx]: 1000*2  target[idx]:24
        emg_data = self.emg_data_list[idx]
        
        emg = emg_data.astype(np.float32)
        
        min_max_data = self.min_max_vals[idx]   
        
        # 记录min, max所在bin的下标，即哪一类
        target,_ = data_utils.min_max_bin_index(min_max_data,self.finger_type_list,self.classify_num)
        target = target.astype(np.int64)
            
        return emg,target

    def get_test(self, idx):
        return self.__getitem__(idx)